# Google Colab으로 오픈소스 LLM 구동하기

## 1단계 - LLM 양자화에 필요한 패키지 설치
- bitsandbytes: Bitsandbytes는 CUDA 사용자 정의 함수, 특히 8비트 최적화 프로그램, 행렬 곱셈(LLM.int8()) 및 양자화 함수에 대한 경량 래퍼
- PEFT(Parameter-Efficient Fine-Tuning): 모델의 모든 매개변수를 미세 조정하지 않고도 사전 훈련된 PLM(언어 모델)을 다양한 다운스트림 애플리케이션에 효율적으로 적용 가능
- accelerate: PyTorch 모델을 더 쉽게 여러 컴퓨터나 GPU에서 사용할 수 있게 해주는 도구


In [1]:
#양자화에 필요한 패키지 설치
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 769.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.tom

## 2단계 - 트랜스포머에서 BitsandBytesConfig를 통해 양자화 매개변수 정의하기


* load_in_4bit=True: 모델을 4비트 정밀도로 변환하고 로드하도록 지정
* bnb_4bit_use_double_quant=True: 메모리 효율을 높이기 위해 중첩 양자화를 사용하여 추론 및 학습
* bnd_4bit_quant_type="nf4": 4비트 통합에는 2가지 양자화 유형인 FP4와 NF4가 제공됨. NF4 dtype은 Normal Float 4를 나타내며 QLoRA 백서에 소개되어 있습니다. 기본적으로 FP4 양자화 사용
* bnb_4bit_compute_dype=torch.bfloat16: 계산 중 사용할 dtype을 변경하는 데 사용되는 계산 dtype. 기본적으로 계산 dtype은 float32로 설정되어 있지만 계산 속도를 높이기 위해 bf16으로 설정 가능



In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## 3단계 - 경량화 모델 로드하기

이제 모델 ID를 지정한 다음 이전에 정의한 양자화 구성으로 로드합니다.

In [3]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

## 4단계 - 잘 실행되는지 확인

In [6]:
device = 'cuda:0'

In [12]:
from transformers import StoppingCriteriaList, MaxLengthCriteria

In [14]:
# Choose your prompt
prompt = "RAG에 대해서 알고 있니? 예시 코드를 작성해볼래?"   # Korean example

messages = [
    {"role": "system",
     "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", truncation=True, padding=True).to(model.device)

attention_mask = inputs.ne(tokenizer.pad_token_id).long()
pad_token_id = tokenizer.eos_token_id

outputs = model.generate(
    inputs,
    attention_mask = attention_mask,
    pad_token_id = pad_token_id,
    max_length=256
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

system

You are a helpful assistant.user

RAG에 대해서 알고 있니? 예시 코드를 작성해볼래?assistant

네, RAG (Recurrent Attention Generator)에 대해 알고 있습니다. RAG는 시퀀스 데이터(텍스트, 이미지 등)에 주석을 추가하는 작업인 시퀀스 투 시퀀스 생성(S2S)에서 사용되는 강력한 모델입니다.

RAG의 핵심 아이디어는 다음과 같습니다:

1. **인코더**: 주어진 시퀀스를 임베딩 벡터로 변환합니다. 이 임베딩 벡터는 시퀀스의 문맥 정보를 담고 있습니다.

2. **어텐션 메커니즘**: 인코더의 각 토큰에 대해 모델은 주석 후보 문장에 대해 어텐션 가중치를 계산합니다. 이 가중치는 주석 생성 시 각 토큰이 주석에 얼마나 중요한지 나타냅니다.

3. **디코더**: 어텐션 가중치를 사용하여 주석을 단어 단위로 생성합니다. 디코더는 이전에 생성된 단어


## 5단계- RAG 시스템 결합하기

In [18]:
# pip install시 utf-8, ansi 관련 오류날 경우 필요한 코드
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [17]:
!pip -q install langchain pypdf chromadb sentence-transformers faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [19]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


In [23]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from transformers import pipeline
from langchain.chains import LLMChain

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=True,
    max_new_tokens=128,
)

prompt_template = """
### [INST]
Instruction: Answer the question based on your knowledge.
Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
 """

koplatyi_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)

Device set to use cuda:0
<ipython-input-23-5cabaf0fc269>:38: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=koplatyi_llm, prompt=prompt)


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.schema.runnable import RunnablePassthrough

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
pip install pypdf

  Using cached pypdf-5.3.1-py3-none-any.whl.metadata (7.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 17.9 MB/s eta 0:00:00


In [31]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 62.5 MB/s eta 0:00:00


In [32]:
loader = PyPDFLoader("/content/drive/MyDrive/강의 자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(texts, hf)
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 3}
                        )

In [33]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


In [34]:
import warnings
warnings.filterwarnings('ignore')

In [35]:
import torch

# CUDA 사용 가능 여부 확인
print("CUDA is available:", torch.cuda.is_available())

# 사용 가능한 GPU 개수 확인
if torch.cuda.is_available():
    print("Number of available GPUs:", torch.cuda.device_count())

    # 현재 사용 중인 GPU 이름 확인
    print("Current GPU:", torch.cuda.get_device_name(torch.cuda.current_device()))

    # CUDA 버전 확인
    print("CUDA version:", torch.version.cuda)

# PyTorch 버전 확인
print("PyTorch version:", torch.__version__)

# 현재 기본 텐서 타입 확인
print("Default tensor type:", torch.get_default_dtype())

# MPS (Mac M1 GPU) 사용 가능 여부 확인 (PyTorch 1.12 이상에서 사용 가능)
if hasattr(torch.backends, 'mps'):
    print("MPS (Mac M1 GPU) is available:", torch.backends.mps.is_available())
else:
    print("MPS (Mac M1 GPU) is not supported in this PyTorch version")

CUDA is available: True
Number of available GPUs: 1
Current GPU: Tesla T4
CUDA version: 12.4
PyTorch version: 2.5.1+cu124
Default tensor type: torch.float32
MPS (Mac M1 GPU) is available: False


In [36]:
result = rag_chain.invoke("혁신성장 정책 금융에서 인공지능이 중요한가?")

for i in result['context']:
    print(f"주어진 근거: {i.page_content} / 출처: {i.metadata['source']} - {i.metadata['page']} \n\n")

print(f"\n답변: {result['text']}")

주어진 근거: 데이터의 효율적 가공과 관리를 위한 인간두뇌와 유사한 형태의 정보처리기술을 말하며, 인공지능, 상황인지컴퓨팅 등의 품목으로 구성됨-컴퓨팅 기술을 활용한 다양한 사업화가 활발히 진행되고 있어 혁신성장 정책금융 공급 규모가 매년 약 100% 수준으로 증가하고 있으며, 새정부의 ‘미래 먹거리산업 신성장 전략추진*’에 따라 인공지능 관련 기술로의 금융지원이 늘어날 것으로 전망됨       * 에너지, 방산, 우주항공, 인공지능(AI), 바이오, 탄소중립 대응, 스마트농업을 차세대 6대 먹거리 산업으로 선정 / 출처: /content/drive/MyDrive/강의 자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf - 3 


주어진 근거: | 8 | CIS이슈리포트 2022-2호 
▶(주요품목② : 인공지능) 정보통신 테마 내 기술분야 중 정책금융 공급규모 증가율이 가장 높은 능동형컴퓨팅 분야의 경우, 인공지능 품목의 정책금융 공급 비중이 가장 높으며, 이는 빅데이터 분석기술의 발전으로 인해 인공지능의 활용처가 넓어짐에 따른 것으로 분석됨[능동형컴퓨팅 분야 내 기술품목별 혁신성장 정책금융 공급액 추이](단위: 억 원)
주: 스마트물류시스템 품목은 2021년부터 신규 품목으로 편임▶인공지능은 인간의 학습능력과 추론·지각능력, 자연언어 이해능력 등을 프로그램으로 구현한 기술로, 컴퓨터가 인간의 지능적인 행동을 모방하는 방향으로 발전하고 있음￮인공지능은 사람의 두뇌가 복잡한 연산을 수행하는 점을 모방해 뉴런(Neuron)을 수학적으로 모방한 알고리즘인 퍼셉트론(Perceptron)을 이용하여 컴퓨터의 연산 로직을 처리하는 원리로 동작함[인공지능 동작 개념]구분 구조뉴런Ÿ세포체의 자극이 임계치를 넘으면 신경전달물질 발화 / 출처: /content/drive/MyDrive/강의 자료/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf